<a href="https://www.kaggle.com/code/i200605salehahmad/100-f1-score-classification-adaboost?scriptVersionId=131127474" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install lazypredict

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

import lazypredict
from lazypredict.Supervised import LazyClassifier

In [ ]:
df = pd.read_csv('/kaggle/input/easiest-diabetes-classification-dataset/Diabetes Classification.csv')
df

In [ ]:
X = df.drop('Diagnosis', axis=1)
Y = df['Diagnosis']

np.shape(X), np.shape(Y)

In [ ]:
#Check for nulls in the dataset
X.isnull().sum()

In [ ]:
AllNumericalCols = [col for col in X.columns if df[col].dtypes!='O']
AllNumericalCols

In [ ]:
AllCategoricalCols = set(X.columns) - set(AllNumericalCols)
AllCategoricalCols

In [ ]:
#NOrmalize numerical columns
scaler = StandardScaler()
X[AllNumericalCols] = scaler.fit_transform(X[AllNumericalCols])
X

In [ ]:
#Label Encode Categorical Columns
LE = LabelEncoder()
for idxcol,col in enumerate(AllCategoricalCols):
    X[col] = LE.fit_transform(X[col])
X

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

np.shape(X_train),np.shape(X_test),np.shape(Y_train),np.shape(Y_test)

In [ ]:
#Call Lazy Predict
LZ = LazyClassifier()
models,predictions = LZ.fit(X_train, X_test, Y_train, Y_test)
models

# As from lazy predict top classifier is Adaboost so, calling ada boost on another set of train and test splits

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

model = AdaBoostClassifier()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

acc = accuracy_score(Y_test, Y_pred)*100
prec = precision_score(Y_test, Y_pred, average='macro')*100
rec = recall_score(Y_test, Y_pred, average='macro')*100
f1 = f1_score(Y_test, Y_pred, average='macro')*100

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", rec)
print("F1 Score: ", f1)